In [1]:
data=[]
import csv
import datetime
import time
import pandas as pd
import numpy as np
import model.vae.vae as vae
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torch.utils.data import TensorDataset
from sklearn import preprocessing


In [2]:
csv_flag='xls'

if csv_flag=="csv":
    f = open('./data/eventlog.csv', 'r')
    with f:
        reader = csv.reader(f)
        for row in reader:
            print(row)
            row[2]=row[2][0:row[2].find(".")]
            timeArray = time.strptime(row[2], "%Y-%m-%d %H:%M:%S")
            timeStamp = int(time.mktime(timeArray))
            row[2]=  str(int(time.mktime(timeArray)))
            data.append([row[2],row[3],row[-1]])
elif csv_flag=="txt":
    ''''''
    with open('./data/eventlog.txt', 'r', newline='') as f:
        reader = f.readlines()
        for row in reader:
            data.append(row)
            print(row)
elif csv_flag=="xls":
    df = pd.read_excel('./data/eventlog.xls')
    for row in df.values:
        timeArray = time.strptime(row[2], "%Y-%m-%d %H:%M:%S")
        timeStamp = int(time.mktime(timeArray))
        row[2]=  str(int(time.mktime(timeArray)))
        data.append(row[1:])


In [3]:
"""data: event category,timestamp,source,ID,type,data"""
print(len(data))
for i in data[0:100]:
    print(i)

3009
[1014 '1653650672' 'Microsoft-Windows-DNS-Client' 1014 2
 'wpad\n128\n02000000C0A85802000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000\n']
[0 '1653650665' 'e1i65x64' 1610874912 4
 '\nIntel(R) 82574L Gigabit Network Connection\n']
[0 '1653650659' 'e1i65x64' -1610350565 2
 '\nIntel(R) 82574L Gigabit Network Connection\n']
[1014 '1653650636' 'Microsoft-Windows-DNS-Client' 1014 2
 'wpad\n128\n02000000C0A85802000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000\n']
[0 '1653650630' 'Microsoft-Windows-Time-Service' 37 4
 'time.windows.com,0x9 (ntp.m|0x9|0.0.0.0:123->20.189.79.72:123)\n']
[0 '1653650628' 'e1i65x64' 

In [4]:
#用于把source变成int
source_dict=dict()
for i in data:
    source_dict[i[2]]=0
for i in data:
    source_dict[i[2]]+=1
print(source_dict)
by_value = sorted(source_dict.items(),key = lambda item:item[1])
#按照出现的频率赋予int
#不同于词向量，我们认为出现频率高的source是安全的
#所以出现频率低的source应该和高频source距离远
#在预测时把没出现过的source置为0
print(by_value)
for i in range(len(by_value)):
    source_dict[by_value[i][0]]=i+1
print(source_dict)

{'Microsoft-Windows-DNS-Client': 520, 'e1i65x64': 132, 'Microsoft-Windows-Time-Service': 58, 'DCOM': 89, 'Service Control Manager': 127, 'Microsoft-Windows-Kernel-General': 650, 'EventLog': 48, 'Display': 432, 'Microsoft-Windows-WindowsUpdateClient': 553, 'Microsoft-Windows-Winlogon': 12, 'Microsoft-Windows-DHCPv6-Client': 18, 'Microsoft-Windows-Dhcp-Client': 36, 'Microsoft-Windows-FilterManager': 108, 'Microsoft-Windows-Directory-Services-SAM': 24, 'Microsoft-Windows-Wininit': 8, 'Microsoft-Windows-Kernel-Processor-Power': 32, 'Microsoft-Windows-Kernel-Power': 15, 'Microsoft-Windows-Ntfs': 8, 'vmci': 9, 'Microsoft-Windows-HAL': 8, 'Microsoft-Windows-Kernel-Boot': 56, 'User32': 5, 'Schannel': 3, 'HTTP': 32, 'Microsoft-Windows-UserModePowerService': 10, 'NetBT': 4, 'Server': 1, 'stornvme': 1, 'Microsoft-Windows-Bits-Client': 1, 'Microsoft-Windows-UserPnp': 5, 'Microsoft-Windows-GroupPolicy': 1, 'Microsoft-Windows-TPM-WMI': 1, 'Virtual Disk Service': 2}
[('Server', 1), ('stornvme', 1), (

In [5]:
ID_dict=dict()
for i in data:
    ID_dict[i[-3]]=0
for i in data:
    ID_dict[i[-3]]+=1
print(ID_dict)#看一下id的分布

{1014: 520, 1610874912: 70, -1610350565: 62, 37: 15, 134: 24, 10016: 70, 1073748864: 88, 16: 548, -2147477635: 24, 35: 10, 1073748869: 30, 4107: 432, 19: 118, 43: 121, 44: 309, -1073734798: 8, 7001: 7, 51046: 8, 50103: 8, 50036: 8, 6: 96, 1: 45, 16983: 5, 16977: 11, 16962: 8, 14: 8, 24: 41, 55: 32, 172: 8, 98: 8, 1074266115: 9, 20: 21, 32: 8, 18: 8, 27: 8, 25: 8, 238: 8, 153: 8, 12: 17, 13: 6, 109: 6, 50037: 5, -2147477643: 8, -2147477639: 8, 50106: 5, 51057: 5, 51047: 5, 50105: 5, 50104: 5, -2147477642: 5, 7002: 5, -2147482574: 5, 36871: 3, 1073756831: 16, 1073756832: 16, 15: 13, 52: 5, 34: 4, -1073737503: 4, -1073739319: 1, 10010: 12, 22: 2, 41: 1, -2147477640: 1, 10001: 7, -2147221375: 1, 16385: 1, 20003: 5, 1501: 1, 1281: 1, -2147477637: 2, 1107296260: 1, 1107296259: 1, -1073734801: 1}


In [6]:
data_pca=[]
for d in data:
    data_str=d[-1]
    handle=[]
    for s in data_str:
        handle.append(ord(s))
    for i in range(400-len(data_str)):
        handle.append(0)
    data_pca.append(handle)
    
for i in data_pca:
    print(i)

[119, 112, 97, 100, 10, 49, 50, 56, 10, 48, 50, 48, 48, 48, 48, 48, 48, 67, 48, 65, 56, 53, 56, 48, 50, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 

[119, 112, 97, 100, 10, 49, 50, 56, 10, 48, 50, 48, 48, 48, 48, 48, 48, 67, 48, 65, 56, 53, 56, 48, 50, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 

[97, 112, 112, 108, 105, 99, 97, 116, 105, 111, 110, 45, 115, 112, 101, 99, 105, 102, 105, 99, 10, 76, 111, 99, 97, 108, 10, 65, 99, 116, 105, 118, 97, 116, 105, 111, 110, 10, 123, 50, 53, 57, 51, 70, 56, 66, 57, 45, 52, 69, 65, 70, 45, 52, 53, 55, 67, 45, 66, 54, 56, 65, 45, 53, 48, 70, 54, 66, 56, 69, 65, 54, 66, 53, 52, 125, 10, 123, 49, 53, 67, 50, 48, 66, 54, 55, 45, 49, 50, 69, 55, 45, 52, 66, 66, 54, 45, 57, 50, 66, 66, 45, 55, 65, 70, 70, 48, 55, 57, 57, 55, 52, 48, 50, 125, 10, 68, 69, 83, 75, 84, 79, 80, 45, 55, 85, 85, 51, 49, 86, 77, 10, 90, 104, 117, 95, 89, 10, 83, 45, 49, 45, 53, 45, 50, 49, 45, 57, 56, 52, 48, 51, 54, 49, 49, 53, 45, 50, 57, 57, 53, 57, 56, 57, 48, 50, 45, 49, 57, 51, 50, 53, 53, 52, 48, 52, 56, 45, 49, 48, 48, 48, 10, 76, 111, 99, 97, 108, 72, 111, 115, 116, 32, 40, 85, 115, 105, 110, 103, 32, 76, 82, 80, 67, 41, 10, 85, 110, 97, 118, 97, 105, 108, 97, 98, 108, 101, 10, 85, 110, 97, 118, 97, 105, 108, 97, 98, 108, 101, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[48, 120, 48, 10, 49, 48, 10, 48, 10, 53, 10, 108, 117, 97, 102, 118, 10, 50, 48, 49, 54, 45, 48, 49, 45, 50, 52, 84, 49, 48, 58, 49, 53, 58, 53, 49, 46, 48, 48, 48, 48, 48, 48, 48, 90, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [7]:
from sklearn.decomposition import PCA
pca = PCA(n_components=10)
new_data_pca = pca.fit_transform(data_pca)

In [8]:
print(new_data_pca)
print(data[0])

[[ 223.49695955  345.33592112   16.45318333 ...  -21.20088162
   -18.11053312   -6.39790691]
 [-393.36121258   -7.93110797 -104.8381776  ...   15.88317949
   -67.67859633  -11.4437482 ]
 [-393.36121258   -7.93110797 -104.8381776  ...   15.88317949
   -67.67859633  -11.4437482 ]
 ...
 [-497.5025951   216.01119887  278.02051409 ...   26.75804067
   -12.06546158   16.34566681]
 [-426.39381869   60.67111049    8.79775722 ...   38.90350111
     5.44663394   34.4996587 ]
 [-442.31390563  121.08284696   66.97392045 ...   22.36712234
    77.67215698  -23.02742455]]
[1014 '1653650672' 'Microsoft-Windows-DNS-Client' 1014 2
 'wpad\n128\n02000000C0A85802000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000\n']


In [9]:
"""data: event category,timestamp,source,ID,type,data"""
data_vae=[]
for d in range(len(data)):
    t=data[d]
    trow=[int(t[0]),int(t[1]),source_dict[t[2]],int(t[3]),int(t[4])]
    trow.extend(new_data_pca[d])
    data_vae.append(trow)

min_max_scaler = preprocessing.MinMaxScaler()
data_vae = min_max_scaler.fit_transform(data_vae)
print(np.shape(data_vae))

(3009, 15)


In [10]:
train=1
if train:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = vae.VAE().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=vae.learning_rate)

    data_loader= TensorDataset(torch.tensor(data_vae),torch.tensor(range(len(data_vae))))
    for epoch in range(vae.num_epochs):
            for i, (x, _) in enumerate(data_loader):
                # Forward pass
                x=x.to(torch.float32)
                x_reconst, mu, log_var = model(x)
                # Compute reconstruction loss and kl divergence
                # For KL divergence, see Appendix B in VAE paper or http://yunjey47.tistory.com/43
                reconst_loss = F.binary_cross_entropy(x_reconst, x, size_average=False)
                kl_div = - 0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())

                # Backprop and optimize
                loss = reconst_loss + kl_div
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                if (i+1) % 100 == 0:
                    print ("Epoch[{}/{}], Step [{}/{}], Reconst Loss: {:.4f}, KL Div: {:.4f}" 
                           .format(epoch+1, vae.num_epochs, i+1, len(data_loader), reconst_loss.item(), kl_div.item()))

Epoch[1/50], Step [100/3009], Reconst Loss: 9.7644, KL Div: 0.0129


C:\Users\wrl\anaconda3\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Epoch[1/50], Step [200/3009], Reconst Loss: 9.2499, KL Div: 0.0166
Epoch[1/50], Step [300/3009], Reconst Loss: 8.3153, KL Div: 0.0251
Epoch[1/50], Step [400/3009], Reconst Loss: 8.1439, KL Div: 0.0415
Epoch[1/50], Step [500/3009], Reconst Loss: 7.9899, KL Div: 0.0153
Epoch[1/50], Step [600/3009], Reconst Loss: 8.1191, KL Div: 0.0061
Epoch[1/50], Step [700/3009], Reconst Loss: 8.1462, KL Div: 0.0092
Epoch[1/50], Step [800/3009], Reconst Loss: 8.9963, KL Div: 0.0055
Epoch[1/50], Step [900/3009], Reconst Loss: 7.9315, KL Div: 0.0035
Epoch[1/50], Step [1000/3009], Reconst Loss: 7.9502, KL Div: 0.0141
Epoch[1/50], Step [1100/3009], Reconst Loss: 7.7062, KL Div: 0.0024
Epoch[1/50], Step [1200/3009], Reconst Loss: 7.6033, KL Div: 0.0086
Epoch[1/50], Step [1300/3009], Reconst Loss: 8.0179, KL Div: 0.0016
Epoch[1/50], Step [1400/3009], Reconst Loss: 7.7627, KL Div: 0.0025
Epoch[1/50], Step [1500/3009], Reconst Loss: 7.8627, KL Div: 0.0044
Epoch[1/50], Step [1600/3009], Reconst Loss: 8.1755, KL 

Epoch[5/50], Step [600/3009], Reconst Loss: 7.6598, KL Div: 0.0010
Epoch[5/50], Step [700/3009], Reconst Loss: 8.0224, KL Div: 0.0009
Epoch[5/50], Step [800/3009], Reconst Loss: 8.8901, KL Div: 0.0005
Epoch[5/50], Step [900/3009], Reconst Loss: 7.6415, KL Div: 0.0009
Epoch[5/50], Step [1000/3009], Reconst Loss: 7.8896, KL Div: 0.0015
Epoch[5/50], Step [1100/3009], Reconst Loss: 7.6226, KL Div: 0.0002
Epoch[5/50], Step [1200/3009], Reconst Loss: 7.4579, KL Div: 0.0006
Epoch[5/50], Step [1300/3009], Reconst Loss: 7.9958, KL Div: 0.0000
Epoch[5/50], Step [1400/3009], Reconst Loss: 7.7186, KL Div: 0.0002
Epoch[5/50], Step [1500/3009], Reconst Loss: 7.9423, KL Div: 0.0012
Epoch[5/50], Step [1600/3009], Reconst Loss: 8.1724, KL Div: 0.0005
Epoch[5/50], Step [1700/3009], Reconst Loss: 7.9947, KL Div: 0.0003
Epoch[5/50], Step [1800/3009], Reconst Loss: 8.9851, KL Div: 0.0009
Epoch[5/50], Step [1900/3009], Reconst Loss: 8.3177, KL Div: 0.0004
Epoch[5/50], Step [2000/3009], Reconst Loss: 8.0146,

Epoch[9/50], Step [800/3009], Reconst Loss: 8.9487, KL Div: 0.0014
Epoch[9/50], Step [900/3009], Reconst Loss: 7.6847, KL Div: 0.0013
Epoch[9/50], Step [1000/3009], Reconst Loss: 7.8648, KL Div: 0.0023
Epoch[9/50], Step [1100/3009], Reconst Loss: 7.6443, KL Div: 0.0017
Epoch[9/50], Step [1200/3009], Reconst Loss: 7.4893, KL Div: 0.0003
Epoch[9/50], Step [1300/3009], Reconst Loss: 8.0749, KL Div: 0.0002
Epoch[9/50], Step [1400/3009], Reconst Loss: 7.7029, KL Div: 0.0001
Epoch[9/50], Step [1500/3009], Reconst Loss: 7.8603, KL Div: 0.0029
Epoch[9/50], Step [1600/3009], Reconst Loss: 8.1438, KL Div: 0.0006
Epoch[9/50], Step [1700/3009], Reconst Loss: 7.9982, KL Div: 0.0012
Epoch[9/50], Step [1800/3009], Reconst Loss: 9.0181, KL Div: 0.0007
Epoch[9/50], Step [1900/3009], Reconst Loss: 8.3619, KL Div: 0.0003
Epoch[9/50], Step [2000/3009], Reconst Loss: 8.0390, KL Div: 0.0002
Epoch[9/50], Step [2100/3009], Reconst Loss: 8.0835, KL Div: 0.0024
Epoch[9/50], Step [2200/3009], Reconst Loss: 7.994

...

Epoch[33/50], Step [1500/3009], Reconst Loss: 7.7680, KL Div: 0.0002
Epoch[33/50], Step [1600/3009], Reconst Loss: 8.2141, KL Div: 0.0013
Epoch[33/50], Step [1700/3009], Reconst Loss: 7.9589, KL Div: 0.0001
Epoch[33/50], Step [1800/3009], Reconst Loss: 9.0463, KL Div: 0.0012
Epoch[33/50], Step [1900/3009], Reconst Loss: 8.3346, KL Div: 0.0001
Epoch[33/50], Step [2000/3009], Reconst Loss: 7.9918, KL Div: 0.0000
Epoch[33/50], Step [2100/3009], Reconst Loss: 8.0294, KL Div: 0.0001
Epoch[33/50], Step [2200/3009], Reconst Loss: 7.9218, KL Div: 0.0000
Epoch[33/50], Step [2300/3009], Reconst Loss: 8.0775, KL Div: 0.0002
Epoch[33/50], Step [2400/3009], Reconst Loss: 8.0145, KL Div: 0.0002
Epoch[33/50], Step [2500/3009], Reconst Loss: 7.6720, KL Div: 0.0002
Epoch[33/50], Step [2600/3009], Reconst Loss: 7.9362, KL Div: 0.0002
Epoch[33/50], Step [2700/3009], Reconst Loss: 7.6602, KL Div: 0.0000
Epoch[33/50], Step [2800/3009], Reconst Loss: 7.7349, KL Div: 0.0012
Epoch[33/50], Step [2900/3009], Re

Epoch[37/50], Step [1500/3009], Reconst Loss: 7.7351, KL Div: 0.0002
Epoch[37/50], Step [1600/3009], Reconst Loss: 8.2134, KL Div: 0.0001
Epoch[37/50], Step [1700/3009], Reconst Loss: 8.0037, KL Div: 0.0000
Epoch[37/50], Step [1800/3009], Reconst Loss: 9.1301, KL Div: 0.0011
Epoch[37/50], Step [1900/3009], Reconst Loss: 8.3844, KL Div: 0.0000
Epoch[37/50], Step [2000/3009], Reconst Loss: 8.1126, KL Div: 0.0000
Epoch[37/50], Step [2100/3009], Reconst Loss: 8.0492, KL Div: 0.0001
Epoch[37/50], Step [2200/3009], Reconst Loss: 7.9799, KL Div: 0.0000
Epoch[37/50], Step [2300/3009], Reconst Loss: 8.0954, KL Div: 0.0002
Epoch[37/50], Step [2400/3009], Reconst Loss: 8.0616, KL Div: 0.0002
Epoch[37/50], Step [2500/3009], Reconst Loss: 7.7403, KL Div: 0.0002
Epoch[37/50], Step [2600/3009], Reconst Loss: 7.9160, KL Div: 0.0001
Epoch[37/50], Step [2700/3009], Reconst Loss: 7.6972, KL Div: 0.0001
Epoch[37/50], Step [2800/3009], Reconst Loss: 7.7362, KL Div: 0.0008
Epoch[37/50], Step [2900/3009], Re

Epoch[41/50], Step [1600/3009], Reconst Loss: 8.2297, KL Div: 0.0002
Epoch[41/50], Step [1700/3009], Reconst Loss: 8.0073, KL Div: 0.0000
Epoch[41/50], Step [1800/3009], Reconst Loss: 9.0688, KL Div: 0.0023
Epoch[41/50], Step [1900/3009], Reconst Loss: 8.3683, KL Div: 0.0000
Epoch[41/50], Step [2000/3009], Reconst Loss: 8.1159, KL Div: 0.0000
Epoch[41/50], Step [2100/3009], Reconst Loss: 8.0701, KL Div: 0.0001
Epoch[41/50], Step [2200/3009], Reconst Loss: 7.9616, KL Div: 0.0000
Epoch[41/50], Step [2300/3009], Reconst Loss: 8.1015, KL Div: 0.0004
Epoch[41/50], Step [2400/3009], Reconst Loss: 8.0607, KL Div: 0.0002
Epoch[41/50], Step [2500/3009], Reconst Loss: 7.7410, KL Div: 0.0002
Epoch[41/50], Step [2600/3009], Reconst Loss: 7.9719, KL Div: 0.0001
Epoch[41/50], Step [2700/3009], Reconst Loss: 7.6918, KL Div: 0.0001
Epoch[41/50], Step [2800/3009], Reconst Loss: 7.7364, KL Div: 0.0002
Epoch[41/50], Step [2900/3009], Reconst Loss: 8.2429, KL Div: 0.0000
Epoch[41/50], Step [3000/3009], Re

Epoch[45/50], Step [1700/3009], Reconst Loss: 7.9985, KL Div: 0.0000
Epoch[45/50], Step [1800/3009], Reconst Loss: 9.1361, KL Div: 0.0008
Epoch[45/50], Step [1900/3009], Reconst Loss: 8.3752, KL Div: 0.0000
Epoch[45/50], Step [2000/3009], Reconst Loss: 8.1146, KL Div: 0.0000
Epoch[45/50], Step [2100/3009], Reconst Loss: 8.0411, KL Div: 0.0000
Epoch[45/50], Step [2200/3009], Reconst Loss: 7.9699, KL Div: 0.0000
Epoch[45/50], Step [2300/3009], Reconst Loss: 8.0795, KL Div: 0.0001
Epoch[45/50], Step [2400/3009], Reconst Loss: 8.0486, KL Div: 0.0000
Epoch[45/50], Step [2500/3009], Reconst Loss: 7.7552, KL Div: 0.0000
Epoch[45/50], Step [2600/3009], Reconst Loss: 7.9405, KL Div: 0.0000
Epoch[45/50], Step [2700/3009], Reconst Loss: 7.6856, KL Div: 0.0000
Epoch[45/50], Step [2800/3009], Reconst Loss: 7.7336, KL Div: 0.0004
Epoch[45/50], Step [2900/3009], Reconst Loss: 8.2285, KL Div: 0.0000
Epoch[45/50], Step [3000/3009], Reconst Loss: 8.5656, KL Div: 0.0000
Epoch[46/50], Step [100/3009], Rec

Epoch[49/50], Step [1700/3009], Reconst Loss: 7.9934, KL Div: 0.0000
Epoch[49/50], Step [1800/3009], Reconst Loss: 9.1380, KL Div: 0.0004
Epoch[49/50], Step [1900/3009], Reconst Loss: 8.3685, KL Div: 0.0000
Epoch[49/50], Step [2000/3009], Reconst Loss: 8.1162, KL Div: 0.0000
Epoch[49/50], Step [2100/3009], Reconst Loss: 8.0021, KL Div: 0.0001
Epoch[49/50], Step [2200/3009], Reconst Loss: 7.9675, KL Div: 0.0000
Epoch[49/50], Step [2300/3009], Reconst Loss: 8.0450, KL Div: 0.0003
Epoch[49/50], Step [2400/3009], Reconst Loss: 8.0300, KL Div: 0.0002
Epoch[49/50], Step [2500/3009], Reconst Loss: 7.7522, KL Div: 0.0002
Epoch[49/50], Step [2600/3009], Reconst Loss: 7.9454, KL Div: 0.0001
Epoch[49/50], Step [2700/3009], Reconst Loss: 7.6451, KL Div: 0.0000
Epoch[49/50], Step [2800/3009], Reconst Loss: 7.7302, KL Div: 0.0002
Epoch[49/50], Step [2900/3009], Reconst Loss: 8.2653, KL Div: 0.0000
Epoch[49/50], Step [3000/3009], Reconst Loss: 8.5648, KL Div: 0.0000
Epoch[50/50], Step [100/3009], Rec

In [11]:
def pre(d:list,vaemodel_path='vae.pth',pcamodel_path='pca.model',min_max_scaler_path='min_max_scaler.model'):
    """data: event category,timestamp,source,ID,type,data"""
    model=torch.load(vaemodel_path)
    import joblib
    pcamodel=joblib.load(pcamodel_path)
    min_max_scaler=joblib.load(min_max_scaler_path)
    if d[2] in source_dict.keys():
        trow=[int(d[0]),int(d[1]),source_dict[d[2]],int(d[3]),int(d[4])]
    else:
        trow=[int(d[0]),int(d[1]),0,int(d[3]),int(d[4])]
    data_str=d[-1]
    handle=[]
    for s in data_str:
        handle.append(ord(s))
    for i in range(400-len(data_str)):
        handle.append(0)
    pca_d=pcamodel.transform([handle])
    trow.extend(pca_d[0])
    trow=min_max_scaler.transform([trow])
    
    x=torch.tensor(trow)
    x=x.to(torch.float32)
    x_reconst, mu, log_var = model(x)
    kl_div = - 0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return kl_div.item()

In [12]:
t=[0,'124124124',"Microsoft-Windows-Kernel-Processor-Power",0,0,"asdf"]
print(pre(t,'vae.pth','pca.model','min_max_scaler.model'))
t=[10,'1653650672','MNS-Client', 1, 78, 'wpad\n128\n02000000C0000000000000000\n']
print(pre(t,'vae.pth','pca.model','min_max_scaler.model'))
t=[1014,'1653650672','Microsoft-WindNS-Client', 1014, 2, 'wpad\n128\n02000000C0A8580200000000000000000\n']
print(pre(t,'vae.pth','pca.model','min_max_scaler.model'))

11.579172134399414
0.16713663935661316
2.5033950805664062e-06


In [13]:
torch.save(model.state_dict(), 'vae.pth')
import joblib
joblib.dump(pca, 'pca.model')
joblib.dump(min_max_scaler, 'min_max_scaler.model')
np.save('source_dict.npy', source_dict) 